<a href="https://colab.research.google.com/github/luguzman/NLP/blob/main/BERT_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 1.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=721d23a8101be74893888b6b19fe57c1bfa557daa4a7700cf07f40b6d7f3a35a
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=c7ff307cc9ec3f96ce916c2bd3aa68ca80ab8d8687c9c7a863a5aeadf30cec89
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=5344cf54ef193276d15af066b214cb9f07d962779386602f6bac402291943b14
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 2.8MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

# Fase 2: Pre Procesado de Datos

## Carga de los ficheros

Cargamos los ficheros de nuestro Google Drive personal

In [ ]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/My Drive/Curso de NLP/BERT/sentiment_data/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

## Pre Procesado

### Limpieza

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenización

Necesitamos crear una capa BERT para tener acceso a los metadatos del tokenizador (como el tamaño del vocabulario).

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer # Instacia que nos permitirá convertir los textos a token
# Una vez teniendo este tokenizador necesitamos info adicional de este como cual es el tamaño del vocabulario,
# convertir todo a minúsculas, etc y por lo tanto aplicaremos la siguiente capa:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",   
                            trainable=False)
# en_uncased: english en minúscula
# L-12: Es la versión sencilla de BERT, para que sea rápido de entrenar vs el L-24
# trainable: Se utiliza para indicar si haremos find tuning, si hay que entrenar algo adicional de los propios pesos

# Extraemos el tamaño del vocabulario(file temporal que genera el hub) a partir del tokenizer. 
# Es más que el tamaño es el diccionario completo.
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# Del mismo modo necesitamos la info de la conversión a minúsculas por parte de BERT.
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)
# Esto es normal que tarde pues con esta versión baja la arquitectura junto con los 110 M de pesos

Solo usamos la primera oración para las entradas BERT, por lo que agregamos el token CLS al principio y el token SEP al final de cada oración.

In [ ]:
# Ahora aparecerán dos token uno "CLS" al inicio que se utiliza para problemas de clasificación
# y un token "SEP" token de separación entre las frases . Por supuesto, en nuestro caso solo vamos a 
# tener una frase  porque estamos llevando a cabo una tarea de clasificación. Por lo que en este caso solo
# rendremos que añadir el token de sepración al final y nada más.
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Creación del dataset

Necesitamos crear las 3 entradas diferentes para cada oración.

In [ ]:
# Una vez tokenizada la frase necesitamos crear 3 tokens para las 3 entradas

# La siguiente función a partir de una lista de palabras tokenizadas con el cls + el separador el 
# tokenizer se encarga de convertir los tokens a id's.
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

# La siguiente función aplicará la máscara correspondiente a los tokens de padding. Basicamente lo que 
# hará es buscar dentro de la lista de tokens la existencia del token 'PAD'. Cuando nosotros nos encontremos 
# con ese token 'PAD' significa que ese es un elemento que no nos interesa. Cunado el token en cuestión
# en una posición no sea el de PAD nos devolverá un 0 y 1's donde no.
def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

# La siguiente función indicará si un token pertenece a la primera frase o a la segunda frase. Esto lo 
# haremos gracias a ubicar el token de separación 'SEP'. Entonces, hasta encontrar el token SEP utilizaremos
# 0´s para indicar que el segmento de frase pertence al primer trozo y posteriormente colocaremos 1´s
def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # convierte los 1 en 0 y vice versa
    return seg_ids

Crearemos padded batches (por lo que rellenamos las frases para cada lote de forma independiente), de esta forma añadimos el mínimo número de tokens de padding posible. Para eso, ordenamos las frases por longitud, aplicamos padded_batches y luego las mezclamos.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
# Hasta ahora tomabamos como una entrada una secuencia de tokens numéricos. Ahora necesitamos 3
# secuencias de tokens. 
sorted_all = [([get_ids(sent_lab[0]),   # idetificadores para la frase
                get_mask(sent_lab[0]),  # máscara para la isma frase 
                get_segments(sent_lab[0])], # segmentos para la propia frase
               sent_lab[1])         # etiqueta para esta frase que acabos de tripiclar su tamaño
              for sent_lab in data_with_len if sent_lab[2] > 7] # Unicamente si la frase tiene más de 7 palabras

In [ ]:
# Una lista es un tipo de iterador de modo que se puede usar como generador de un dataset.

# En este punto nuestras aun no tiene todas la misma longitud por lo que utilizaremos un generator,
# para que se encargue de arreglarlas. Un generedor como funciona es simplemente le das un elemnto 
# y te duevuelve otro. 
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32)) # Tipo de dato de la salida (enteros en este caso)

In [ ]:
# Ahora haremos el proceso de Padding pero recordemos que será por bloques así reduciremos el entrenamiento
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
# Generamos nustro conjunto de entrenamiento y testing por lotes.

# Declaramos el número total de lotes que va a haber.
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
# Declaramos el número total de lotes de test que va a haber.
NB_BATCHES_TEST = NB_BATCHES // 10  # Nos quedamos con el 10%
# Solo recordemos que los lotes los tenemos ordenados por lo que si nos tomamos el 90% priemro para 
# entrenar y el 10% restante para testing me van a quedar el 10% más largo. Por lo que volveremos a 
# mezclar cada uno de los lotes.
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
next(iter(train_dataset))

(<tf.Tensor: shape=(32, 3, 10), dtype=int32, numpy=
 array([[[  101, 27427,  2229, 23773,  2003,  2025,  2652,  3835,  2651,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101,  1005,  1055,  4439,  9450,  2003,  5881,  1999,  4026,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101,  6289, 23644,  2085,  1045,  2131,  2009,   999,   999,
            102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1,
              1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,
              0]],
 
        [[  101, 24471,  2290,   999,  1045,  3335,  4757,  2017,  1012,
            102],
         [    1,     1,     1

In [ ]:
# Ejemplo
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
print(my_sent)

NameError: ignored

In [ ]:
# Ejemplo de lo que le pasaríamos a la capa de BERT
# necesitamos que se un tensor por lo que expandimos la 1°,2° Y 3° capa de BERT
bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0), 
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0)])

# NOTA: cuando necesitemos utilizar la representación vectorial de una frase, usaremos el primer tensor
# y cuando necesitemos la representación vectorial de cada una de las palabras de la frase utilizaremos
# el segundo tensor.

NameError: ignored

# Fase 3: Construcción del modelo

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    # En este caso la capa de BERT ya sabe cual es la dimensión del embedding dependiendo de cual llamemos y
    # también sabe el tamaño del vocabulario porque utiliza su propio tokenizador por lo que ya no usaremos
    # vocab_size ni emb_dim.
    def __init__(self,
                 nb_filters=50, # Número de filtros que le aplicaremos a RNC por cada vector de características
                 FFN_units=512, # Número de neuronas en la capa oculta. En este caso la penultima capa densa
                 nb_classes=2,  # Ya que es un proceso de clasificación binario
                 dropout_rate=0.1,  # Apaga el 10% de las neuronas de manera aleatoria en cada epoch para evitar el overfitting
                 name="dcnn"):
        # Para poder utlizar todas las varibales de tk.keras.Model lo primero y más importante tenemos que 
        # llamar a la super clase, ya que como hemos hecho que el método herede de tf.keras.Model. estamos 
        # obligados a llamar a :
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        # Ahora la 1° capa de embeddings ya no es la de tensorflow, sino que en este caso utilizamos al
        # muy parecido a lo que utilizamos con la capa de BERT para el tokenizador. Usamos nuevamente
        # hub.KerasLayer y vovlemos a utilzar el modelos simple de 12 capas incrustadas en un espacio de 
        # dimensión 768
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)
        # trainable = False: Porque en este caso quiero utilizar el embedding ya entrenado por Google y
        # no quiero hacer find tuning.

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,      # analizamos las palabras cuando aparecen juntas de dos en dos, ie, filtrará palabras de 2 en 2.
                                    padding="valid",    # aqui no toma mucha importancia, lo que hará es añadir ceros en el caso de las primeras y ultimas convoluciones en las que ya no haya elementos. Pero como no hemos puesto el parametro de stride es = 1 por lo que no se podría salir
                                    activation="relu")  # trabaja muy bien en este tipo de análisis. La aplicamos para romper la linealidad y que busque más allá de valores lineales.  
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        # Creamos una capa la cual se encarga de solo quedarse con el máximo de todo estos valores. El 
        # máximo de cada bi-tri-cuatrigrama
        self.pool = layers.GlobalMaxPool1D()

        # Pasamos estos maximos a la primera capa densa
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    # En la sig función recibe el token de entrada, recordemos que vamos a usar bloques ded tokens y por
    # tanto la capa bert_layer que declaramos en el __init__ la aplicamos aqui y le damos la forma
    # correcta, porque para cada batch de tokens hay obtener los identificadores, las máscaras y el token
    # de frase en el orden adecuado. 
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :], # A todas las frases que vengan en el batch a todas y cada una de las palabras (están en la posición 0 del tensor) en dicha frase 
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        # Recordemos que la respuesta el 1° tensor es para cuando queremos un proceso de análisis de la frase 
        # en global (vector de dim 768 que represnta la frase en todo su contexto) y la 2° corresponde a los 
        # tokens, las palabras individuales mapeadas a ese espacio de 768. Lo que quiero es analizar palabra
        # por palabra.  
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)    # Se ecarga de crear la lista larga con cada uno de los tokens que corresponda, uno detrás del otro

        x_1 = self.bigram(x)    # (batch_size, nb_filters, seq_len-1) Aplicará la transformación de bigramas a x
        x_1 = self.pool(x_1)    # (batch_size, nb_filters) Ya que queremos quedarnos con el más grande, el más importante de esos valores de cada uno de los 50 de los mapas de características
        x_2 = self.trigram(x)   # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2)    # (batch_size, nb_filters)
        x_3 = self.fourgram(x)  # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3)    # (batch_size, nb_filters)
        
        # Concatenamos la lista formada por x1,x2 y x3 para formar nuestra macro entrada que pasará a nuestra red neuronal
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        # -1: representa el último de los ejes de combinacion. En este caso tenemos 2 ejes.
        # El primer eje es el de los bloques:
        # Recordemos que le damos batches de palabras. Por lo tanto el primer eje sería cada uno de los
        # objetos es un comentario.El segundo eje representa el propio valor que acabamos de sacar después
        # de cada uno de los max pooling.
        # Lo que se obtiene en la variable merge será un tensor cuyo tamaño será el número de filas, el
        # mismo que el del tamaño del bloque (batchsize), y el número de columnas será 3 veces * el numero filtros
        # 3 veces se debe a que habrá el número de filtros en bigramas, el número de filtros de trigramas, etc
        
        # Aplicamos la tranformación de la capa densa número 1 a toda esta info ya combinada
        merged = self.dense_1(merged)
        # Aplicamos la capa de Dropout unica y exclusivamente en fase de training 
        merged = self.dropout(merged, training)
        # Por último aplicamos la capa de salida 
        output = self.last_dense(merged)
        
        return output

# Fase 4: Entrenamiento

In [ ]:
NB_FILTERS = 100    # Número de filtros de la red neuronal convolucional
FFN_UNITS = 256     # Número de unidades que la capa de Feed Forward tendrá en la capa oculta
NB_CLASSES = len(set(train_labels))      

DROPOUT_RATE = 0.2  # Definimos la tasa de olvido

BATCH_SIZE = 32
NB_EPOCHS = 5       # Número de veces que pasarémos por todo el conjunto de entrenamiento

In [ ]:
# Creamos la instancia del objeto DCNN
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
# Crearemos un sistema de checkpoint en google colab que en caso de que se reinicie la sesión podemos reanudar
# desde elúltimo checkpoint que se haya guardado o incluso más adelante , seguir entrenando con más textos 
# desde donde lo habíamos dejado en lugar de tener que crear la red neuronal desde el inicio.

# Definimos una ruta dentro de mi Google Drive donde se guardarán los checkpoints
checkpoint_path = "./drive/My Drive/Curso NLP/BERT/ckpt_bert_embedding/"

# Guardamos el modelo que queremos guradar autoamticamente siempre que sea posible
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

# Definimos el manager que será el encargado de guardar los checkpoints en la ruta establecida
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
# max_to_keep: Es para definir que queremos que siempre se guarden los últimos 5 checkpoints.
# En este caso si quisieramso guardar todos bastaría con poner max_to_keep = 5 pues son 5 epochs

# Las siguientes líneas de código lo que hacen es preguntarle al Checkpoint Manager si hay o no hay 
# último checkpoint
if ckpt_manager.latest_checkpoint:  # esta linea devuelve un None si no hay checkpoint previo (If None:)
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
# Función que entrega al método fit. Como se puede apreciar se le pueden entregar uno o más callbacks 
# y el objetivo es que se ejecuten algunas líneas de código entre cada epoch, al inicio de un epoch,
# al final, etc. Hay una serie de metodo que podemos sobreescribir. En este caso on_epoch_end, cuando 
# termine un epoch lo que quiero es que se ejecute el guardado que hemos dicho en el checkpoint manager.

# En otras palabras para que cada que fianlice un epoch haya un guardado de los pesos, de esos parámetros
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint guardado en {}.".format(checkpoint_path))
# logs=None: No queremos que se muestre ningún log en particular 

## Result

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])    # En este caso solo llamamos un callback, sin embargo, en esta lista podríamos llamar a 2,3, etc.

Epoch 1/5
  40623/Unknown - 1579s 39ms/step - loss: 0.3969 - accuracy: 0.8222Checkpoint guardado en ./drive/My Drive/Curso de NLP/BERT/ckpt_bert_embedding/.
40623/40623 [==============================] - 1582s 39ms/step - loss: 0.3969 - accuracy: 0.8222
Epoch 2/5
40623/40623 [==============================] - 1588s 39ms/step - loss: 0.3744 - accuracy: 0.8344
Epoch 3/5
40623/40623 [==============================] - 1613s 40ms/step - loss: 0.3642 - accuracy: 0.8398
Epoch 4/5
40623/40623 [==============================] - 1611s 40ms/step - loss: 0.3565 - accuracy: 0.8435
Epoch 5/5
40623/40623 [==============================] - 1608s 40ms/step - loss: 0.3502 - accuracy: 0.8464


# Stage 5: Evaluation

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 99s 22ms/step - loss: 0.3351 - accuracy: 0.8573
[0.33505797386169434, 0.857308030128479]


Hay que pensar que hay una gran parte de la red neuronal que ahora mismo no ha sido optimizada para absolutamente nada. La red neuronal anteiror, optimizaba los valores edel embedding para que fueran los mejores con base al data set, al diccionario de palabras del que formaba parte. Ahora no hemos congelado esa parte y aún así hemos visto uno mejora no muy significativa pero interesante ya que se ocupo mitad de tiempo, y también perdimos overfitting. 

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    # Recordemos que cuando entrenamos el modelo nostros le pasabamos bloques de frase y si me quedo 
    # unicamente con con lo de arriba no sería un bloque por lo añadimos una dimesnion adiconal

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simula un lote

    output = Dcnn(inputs, training=False)

    # El sentimiento lo obtendremos de redondear a la baja el output * 2. Esto ya que el número esta entre 0
    # y 1 y este al multiplicarlo * 2 estará entre 0 y 2. Al redondear a la baja toda cosa entre 0 y .999
    # será redondeado a 0 y todo valor entre 1 y 1.999 será 1 es imposible obtener 2 así que no hay problema
    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Salida del modelo: {}\nSentimiento predicho: Negativo.".format(
            output))
    elif sentiment == 1:
        print("Salida del modelo: {}\nSentimiento predicho: Positivo.".format(
            output))

In [ ]:
get_prediction("This actor is a deception.")

Salida del modelo: [[0.21923825]]
Sentimiento predicho: Negativo.
